In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import EvaluationModule
import ModelModule.models
import pickle
import sklearn 
from sklearn.model_selection import (KFold, RepeatedKFold,
                                     RepeatedStratifiedKFold, ShuffleSplit,
                                     StratifiedKFold)

In [8]:


param_search_grid={
"Random forest": {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
 },

"XGBoost":dict(
eta =np.arange(.1,1.1,.1),
gamma =np.arange(1,10,1), 
max_depth=np.arange(1,20,2),
min_child_weight= np.arange(1,10,1),
max_delta_step= np.arange(1,10,1),
reg_lambda = np.arange(.1,1.1,.1),
colsample_bytree= np.arange(0.4, 1.0, 0.1),
colsample_bylevel= np.arange(0.4, 1.0, 0.1),
n_estimators=np.arange(100, 1100, 100),
tree_method =['auto', 'exact', 'approx', 'hist', 'gpu_hist'],
)
,
"Linear": dict(
normalize=[True,False],
fit_intercept=[True,False],
)
,
"Ridge": dict(
alpha=[0,1,5,10,20,25,50,100],
normalize=[True,False],
fit_intercept=[True,False],
solver=['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga','lbfgs']
),
"Lasso": dict(
alpha=[0,1,5,10,20,25,50,100],
max_iter=[500,750,1000,1250,1500,2000],
normalize=[True,False],
fit_intercept=[True,False],
selection=['cyclic', 'random']
)
,



# "SVM": dict(
# C = np.logspace(-1, 1, 3),
# kernel=[ 'poly', 'rbf', 'sigmoid'] ,
# gamma = np.logspace(-1, 1, 3),
# shrinking=[True,False]

# )
#,
"ANN": dict(
solver=['lbfgs', 'sgd', 'adam'],
batch_size = [10, 20, 40, 60, 80, 100],
learning_rate= ['constant','adaptive'], 
hidden_layer_sizes= [(50,50,50), (50,100,50), (100,)],
activation=['identity', 'logistic', 'tanh', 'relu'],
alpha=[0.0001,0.001, 0.01, 0.1, 0.2, 0.3]
)




}
dataset_params=[
        # dict(
        # library_root="../Dataset/",
        # challenge='regression', #option: regression, regression_delta, classification, binary_classification
        # dataset='CORRONA CERTAIN', 
        # process_approach='SC', #option: KVB, SC
        # imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
        # patient_group=['bionaive TNF'], #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
        # drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
        # time_points=(0,3), 
        # train_test_rate=0.8,
        # remove_low_DAS = True,
        # save_csv=True,
        # random_state=2022),
         dict(
        library_root="../Dataset/",
        challenge='regression_delta', #option: regression, regression_delta, classification, binary_classification
        dataset='CORRONA CERTAIN', 
        process_approach='SC', #option: KVB, SC
        imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
        patient_group=['bionaive TNF'], #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
        drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
        time_points=(0,3), 
        train_test_rate=0.8,
        remove_low_DAS = True,
        save_csv=True,
        random_state=2022),
        dict(
        library_root="../Dataset/",
        challenge='regression_delta_binary', #option: regression, regression_delta, classification, binary_classification
        dataset='CORRONA CERTAIN', 
        process_approach='SC', #option: KVB, SC
        imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
        patient_group=['bionaive TNF'], #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
        drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
        time_points=(0,3), 
        train_test_rate=0.8,
        remove_low_DAS = True,
        save_csv=True,
        random_state=2022)
        ]
base_estimators={
"Random forest":dict(model_id="Random forest", model_list=["Random Forest"], challenge="Regression"),
"XGBoost":dict(model_id="XGBoost", model_list=["XGBoost"], challenge="Regression"),
"Linear":dict(model_id="Linear", model_list=["Linear"], challenge="Regression"),
"Lasso":dict(model_id="Lasso", model_list=["Lasso"], challenge="Regression"),
"Ridge":dict(model_id="Ridge", model_list=["Ridge"], challenge="Regression"),



#"SVM":dict(model_id="SVM", model_list=["SVM"], challenge="Regression"),

"ANN":dict(model_id="ANN", model_list=["ANN"], challenge="Regression")


}
aml=ModelModule.models.tune_models(dataset_parms=dataset_params,fixed_model_params=base_estimators, test_model_parms=param_search_grid, method='random', project_name="regression_testing",ballance_class=[2])

base Random ForestDone
best_params: {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}
base XGBoostDone
best_params: {'tree_method': 'hist', 'reg_lambda': 0.7000000000000001, 'n_estimators': 300, 'min_child_weight': 4, 'max_depth': 1, 'max_delta_step': 6, 'gamma': 7, 'eta': 0.7000000000000001, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5}
base LinearDone
best_params: {'normalize': False, 'fit_intercept': True}
base LassoDone
best_params: {'selection': 'random', 'normalize': True, 'max_iter': 1000, 'fit_intercept': True, 'alpha': 25}
base RidgeDone
best_params: {'solver': 'lsqr', 'normalize': True, 'fit_intercept': True, 'alpha': 10}
base ANNDone
best_params: {'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (100,), 'batch_size': 40, 'alpha': 0.3, 'activation': 'tanh'}
base Random ForestDone
best_params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_feature

In [10]:
param_search_grid={
"Random forest": {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
 },

"XGBoost":dict(
eta =np.arange(.1,1.1,.1),
gamma =np.arange(1,10,1), 
max_depth=np.arange(1,20,2),
min_child_weight= np.arange(1,10,1),
max_delta_step= np.arange(1,10,1),
reg_lambda = np.arange(.1,1.1,.1),
colsample_bytree= np.arange(0.4, 1.0, 0.1),
colsample_bylevel= np.arange(0.4, 1.0, 0.1),
n_estimators=np.arange(100, 1100, 100),
tree_method =['auto', 'exact', 'approx', 'hist', 'gpu_hist'],
),
"Logistic": dict(
penalty=['l1', 'l2', 'elasticnet', 'none'],
dual=[True,False],
C = np.logspace(-4, 4, 20),   
max_iter = [100, 1000,2500, 5000],
fit_intercept=[True,False],
solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
#),
# "SVM": dict(
# C = np.logspace(-4, 4, 20),
# kernel=['linear', 'poly', 'rbf', 'sigmoid'] ,
# gamma =[1, 0.1, 0.01, 0.001, 0.0001],
# shrinking=[True,False]

# ),
"ANN": dict(
solver=['lbfgs', 'sgd', 'adam'],
batch_size = [10, 20, 40, 60, 80, 100],
learning_rate= ['constant','adaptive'], 
hidden_layer_sizes= [(50,50,50), (50,100,50), (100,)],
activation=['identity', 'logistic', 'tanh', 'relu'],
alpha=[0.0001,0.001, 0.01, 0.1, 0.2, 0.3]


)




}
dataset_params=[
        dict(
        library_root="../Dataset/",
        challenge='classification', #option: regression, regression_delta, classification, binary_classification
        dataset='CORRONA CERTAIN', 
        process_approach='SC', #option: KVB, SC
        imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
        patient_group=['bionaive TNF'], #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
        drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
        time_points=(0,3), 
        train_test_rate=0.8,
        remove_low_DAS = True,
        save_csv=True,
        random_state=2022),
         dict(
        library_root="../Dataset/",
        challenge='binary_classification', #option: regression, regression_delta, classification, binary_classification
        dataset='CORRONA CERTAIN', 
        process_approach='SC', #option: KVB, SC
        imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
        patient_group=['bionaive TNF'], #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
        drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
        time_points=(0,3), 
        train_test_rate=0.8,
        remove_low_DAS = True,
        save_csv=True,
        random_state=2022)
        ]
base_estimators={
"Random forest":dict(model_id="Random forest", model_list=["Random Forest"], challenge="Classification"),
"XGBoost":dict(model_id="XGBoost", model_list=["XGBoost"], challenge="Classification"),
"Logistic":dict(model_id="Logistic", model_list=["Logistic"], challenge="Classification"),
#"SVM":dict(model_id="SVM", model_list=["SVM"], challenge="Classification"),
"ANN":dict(model_id="ANN", model_list=["ANN"], challenge="Classification")


}
aml=ModelModule.models.tune_models(dataset_parms=dataset_params,fixed_model_params=base_estimators, test_model_parms=param_search_grid, method='random', project_name="classfication_testing",ballance_class=[0,1,2])

base Random ForestDone
best_params: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': True}
base XGBoostDone
best_params: {'tree_method': 'auto', 'reg_lambda': 0.6, 'n_estimators': 200, 'min_child_weight': 6, 'max_depth': 5, 'max_delta_step': 5, 'gamma': 3, 'eta': 0.6, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.4}
base LogisticDone
best_params: {'solver': 'newton-cg', 'penalty': 'none', 'max_iter': 100, 'fit_intercept': False, 'dual': False, 'C': 545.5594781168514}
base ANNDone
best_params: {'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50, 50, 50), 'batch_size': 100, 'alpha': 0.1, 'activation': 'tanh'}
base Random ForestDone
best_params: {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}
base XGBoostDone
best_params: {'tree_method': 'auto', 'reg_lambda': 0.8, 'n_estimators': 100, 'min_child_weight': 6, 'ma

In [ ]:
meta_learners=[ "Bayes", 'Logistic', "SVM", 'ANN','XGBoost','Random Forest', 'Tree']
from itertools import chain, combinations
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r+1) for r in range(len(s)))
estemators=[list(c) for c in list(powerset(meta_learners))]

In [ ]:
aml=EvaluationModule.AutoBuild(seed=1,challenge='binary_classification')
parameters= {'ensamble_type':["Stacking"],'model_list':estemators, 'challange':['Classification'], 'meta_learner':meta_learners}
ensamble_model.test_hyper_parameters(parameters,train,test,dataset,aml)